In [ ]:
from google.colab import files


uploaded = files.upload()


Saving final_flight_data (2).csv to final_flight_data (2).csv


In [ ]:
!pip install tensorflow
!pip install tensorflow-decision-forests
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 708.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 876.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Su

In [ ]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os
import math


In [ ]:
# Load your dataset
flight_df = pd.read_csv('final_flight_data (2).csv')
flight_df.head(5)

,Departure Airport Name,Departure Airport ID,Arrival Airport Name,Arrival Airport ID,Duration,Airline,Travel Class,Total Duration,Layover 1 Duration,Layover 1 Overnight,...,Final Arrival Airport ID,Airport Distance (km),Flight Type,Airline Star Rating,Time of Day,Total_Baggage_kg,Departure Time_Date,Departure Time_Time,Arrival Time_Date,Arrival Time_Time
0,Yangon International Airport,RGN,Suvarnabhumi Airport,BKK,85,Myanmar National Airlines,Economy,85,0,False,...,BKK,610.219945,Direct,3,Afternoon,37,20240701,1725,20240701,1920
1,Yangon International Airport,RGN,Suvarnabhumi Airport,BKK,85,Myanmar National Airlines,Economy,85,0,False,...,BKK,610.219945,Direct,3,Morning,37,20240701,745,20240701,940
2,Yangon International Airport,RGN,Suvarnabhumi Airport,BKK,90,THAI,Economy,90,0,False,...,BKK,610.219945,Direct,4,Morning,53,20240701,1135,20240701,1335
3,Yangon International Airport,RGN,Suvarnabhumi Airport,BKK,90,THAI,Economy,90,0,False,...,BKK,610.219945,Direct,4,Evening,53,20240701,1900,20240701,2100
4,Yangon International Airport,RGN,Kuala Lumpur International Airport,KUL,175,Malaysia Airlines,Economy,670,355,False,...,BKK,610.219945,Layover,4,Morning,30,20240701,1125,20240701,1550


In [ ]:
# Name of the label column.
label = "Price"
classes = flight_df[label].unique().tolist()
print(f"Label classes: {classes}")

flight_df[label] = flight_df[label].map(classes.index)

Label classes: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218,

In [ ]:
# Ensure all boolean columns are already in numeric format (1 for True, 0 for False)
boolean_columns = [
    'Layover 1 Overnight', 'Cabin Baggage', 'Checked Baggage', 'Meal Included',
    'Departure Country Is Holiday', 'Arrival Country Is Holiday', 'Overnight Flight'
]

# Convert boolean columns to integers (if necessary)
for col in boolean_columns:
    flight_df[col] = flight_df[col].astype(int)

In [ ]:
# Split the dataset into a training and a testing dataset.
def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

train_ds_pd, test_ds_pd = split_dataset(flight_df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

66145 examples in training, 28692 examples for testing.


In [ ]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)

In [ ]:
model = tfdf.keras.RandomForestModel(
    num_threads=os.cpu_count(),
    task = tfdf.keras.Task.REGRESSION)
model.fit(train_ds)
print(model.summary())

Use /tmp/tmp494qdgqf as temporary training directory
Reading training dataset...
Training dataset read in 0:00:02.024865. Found 66145 examples.
Training model...
Model trained in 0:00:13.573997
Compiling model...
Model compiled.
Model: "random_forest_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: REGRESSION
Label: "__LABEL"

Input Features (32):
	Airline
	Airline_Star_Rating
	Airport_Distance_(km)
	Arrival_Airport_ID
	Arrival_Airport_Name
	Arrival_Country_Is_Holiday
	Arrival_Time_Date
	Arrival_Time_Time
	Cabin_Baggage
	Cabin_Baggage_(Number)
	Cabin_Baggage_(kg_per_piece)
	Checked_Baggage
	Checked_Baggage_(Number)
	Checked_Baggage_(kg_per_piece)
	Day_of_Week_(on_flight_date)
	Departure_Airport_ID
	Departure_

In [ ]:
# Baseline Evaluation
model.compile(metrics=["mse"])
evaluation_original = model.evaluate(test_ds, return_dict=True)
rmse_original = math.sqrt(evaluation_original['mse'])
print(f"Original RMSE: {rmse_original}")


29/29 [==============================] - 3s 92ms/step - loss: 0.0000e+00 - mse: 25217.6660
Original RMSE: 158.80071163450432


In [ ]:
#Start by adding polynomial features, log transformations, or binning and compare the results.
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import KBinsDiscretizer

# Create Polynomial Features
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
poly_features = poly.fit_transform(flight_df.select_dtypes(include=['number']))

# Apply Log Transformation
log_transformer = FunctionTransformer(np.log1p, validate=True)
log_transformed_features = log_transformer.fit_transform(flight_df.select_dtypes(include=['number']))

# Binning
binner = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
binned_features = binner.fit_transform(flight_df.select_dtypes(include=['number']))

# Add these engineered features back to the DataFrame
poly_feature_names = poly.get_feature_names_out(flight_df.select_dtypes(include=['number']).columns)
flight_df_poly = pd.DataFrame(poly_features, columns=poly_feature_names)
flight_df_log = pd.DataFrame(log_transformed_features, columns=[f"log_{col}" for col in flight_df.select_dtypes(include=['number']).columns])
flight_df_binned = pd.DataFrame(binned_features, columns=[f"bin_{col}" for col in flight_df.select_dtypes(include=['number']).columns])

flight_df = pd.concat([flight_df, flight_df_poly, flight_df_log, flight_df_binned], axis=1)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(


In [ ]:
print(flight_df.dtypes)


Departure Airport Name      object
Departure Airport ID        object
Arrival Airport Name        object
Arrival Airport ID          object
Duration                     int64
                            ...   
bin_Total_Baggage_kg       float64
bin_Departure Time_Date    float64
bin_Departure Time_Time    float64
bin_Arrival Time_Date      float64
bin_Arrival Time_Time      float64
Length: 330, dtype: object


In [ ]:
flight_df.columns = [col.replace('[', '_').replace(']', '_').replace(' ', '_') for col in flight_df.columns]


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode any object columns
label_encoders = {}
for col in flight_df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    flight_df[col] = le.fit_transform(flight_df[col])
    label_encoders[col] = le  # Store the encoder for inverse transform if needed


In [ ]:
print(flight_df.dtypes)


Departure_Airport_Name       int64
Departure_Airport_ID         int64
Arrival_Airport_Name         int64
Arrival_Airport_ID           int64
Duration                     int64
                            ...   
bin_Total_Baggage_kg       float64
bin_Departure_Time_Date    float64
bin_Departure_Time_Time    float64
bin_Arrival_Time_Date      float64
bin_Arrival_Time_Time      float64
Length: 330, dtype: object


In [ ]:
print(type(train_ds_pd_updated))
print(type(test_ds_pd_updated))


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [ ]:
print(train_ds_pd_updated.columns)


Index(['Departure_Airport_Name', 'Departure_Airport_ID',
       'Arrival_Airport_Name', 'Arrival_Airport_ID', 'Duration', 'Airline',
       'Travel_Class', 'Total_Duration', 'Layover_1_Duration',
       'Layover_1_Overnight',
       ...
       'bin_Departure_Country_Is_Holiday', 'bin_Arrival_Country_Is_Holiday',
       'bin_Overnight_Flight', 'bin_Airport_Distance__km_',
       'bin_Airline_Star_Rating', 'bin_Total_Baggage_kg',
       'bin_Departure_Time_Date', 'bin_Departure_Time_Time',
       'bin_Arrival_Time_Date', 'bin_Arrival_Time_Time'],
      dtype='object', length=330)


In [ ]:
# Check if the DataFrame is a pandas DataFrame
print(f"Type of train_ds_pd_updated: {type(train_ds_pd_updated)}")
print(f"DataFrame columns: {train_ds_pd_updated.columns}")
print(f"DataFrame dtypes:\n{train_ds_pd_updated.dtypes}")


Type of train_ds_pd_updated: <class 'pandas.core.frame.DataFrame'>
DataFrame columns: Index(['Departure_Airport_Name', 'Departure_Airport_ID',
       'Arrival_Airport_Name', 'Arrival_Airport_ID', 'Duration', 'Airline',
       'Travel_Class', 'Total_Duration', 'Layover_1_Duration',
       'Layover_1_Overnight',
       ...
       'bin_Departure_Country_Is_Holiday', 'bin_Arrival_Country_Is_Holiday',
       'bin_Overnight_Flight', 'bin_Airport_Distance_(km)',
       'bin_Airline_Star_Rating', 'bin_Total_Baggage_kg',
       'bin_Departure_Time_Date', 'bin_Departure_Time_Time',
       'bin_Arrival_Time_Date', 'bin_Arrival_Time_Time'],
      dtype='object', length=330)
DataFrame dtypes:
Departure_Airport_Name       int64
Departure_Airport_ID         int64
Arrival_Airport_Name         int64
Arrival_Airport_ID           int64
Duration                     int64
                            ...   
bin_Total_Baggage_kg       float64
bin_Departure_Time_Date    float64
bin_Departure_Time_Time    floa

In [ ]:
import re

# Clean column names to ensure compatibility
train_ds_pd_updated.columns = [re.sub('[^a-zA-Z0-9_]', '_', col) for col in train_ds_pd_updated.columns]
test_ds_pd_updated.columns = [re.sub('[^a-zA-Z0-9_]', '_', col) for col in test_ds_pd_updated.columns]

# # Reset index if necessary
# train_ds_pd_updated = train_ds_pd_updated.reset_index(drop=True)
# test_ds_pd_updated = test_ds_pd_updated.reset_index(drop=True)


In [ ]:
import tensorflow_decision_forests as tfdf
import pandas as pd
import tensorflow as tf

# Assume train_poly_pd and test_poly_pd are your pandas DataFrames
# and 'price' is your target column

label = 'Price'  # Replace with your actual label column name

# Verify the label column exists
if label not in train_poly_pd.columns:
    raise ValueError(f"The label column '{label}' does not exist in the training DataFrame.")

if label not in test_poly_pd.columns:
    raise ValueError(f"The label column '{label}' does not exist in the testing DataFrame.")

# Optionally, rename columns to ensure compatibility
train_poly_pd.columns = [col.replace(' ', '_').replace('-', '_') for col in train_poly_pd.columns]
test_poly_pd.columns = [col.replace(' ', '_').replace('-', '_') for col in test_poly_pd.columns]

# Convert the pandas DataFrame to a TF-DF dataset
train_poly_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
    train_poly_pd,
    label=label,
    task=tfdf.keras.Task.REGRESSION  # Use the appropriate task
)

test_poly_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
    test_poly_pd,
    label=label,
    task=tfdf.keras.Task.REGRESSION
)


NameError: name 'train_poly_pd' is not defined

In [ ]:
# Split the updated dataset into training and testing datasets again
train_ds_pd_updated, test_ds_pd_updated = split_dataset(flight_df)

# Convert to TensorFlow datasets
train_ds_tf = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd_updated, label="Price", task=tfdf.keras.Task.REGRESSION)
test_ds_tf = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd_updated, label="Price", task=tfdf.keras.Task.REGRESSION)

# Train the model with the updated features
model_updated = tfdf.keras.RandomForestModel(num_threads=os.cpu_count(), task=tfdf.keras.Task.REGRESSION)
model_updated.fit(train_ds_tf)

# Evaluate the model on the test dataset with updated features
evaluation_updated = model_updated.evaluate(test_ds_tf, return_dict=True)
rmse_updated = math.sqrt(evaluation_updated['mse'])
print(f"Updated RMSE with Polynomial Features, Log Transformations, and Binning: {rmse_updated}")

# Compare results
print(f"Improvement in RMSE: {rmse_original - rmse_updated}")


AttributeError: 'DataFrame' object has no attribute 'dtype'